# Sesión 2.3.1 - Procesamiento de texto con la librería scikit-learn
En esta sesión de prácticas vamos a ver cómo podemo usar las funciones de tratamiento de texto de la librería scikit-learn para calcular TF, TF-IDF y BM25.

Primero, instalaremos la librerías necesarias.

In [ ]:
!pip3 install -U scikit-learn
# Descargamos un fichero python con la implementación del BM25
!wget --no-check-certificate https://valencia.inf.um.es/dlpln/BM25.py

# Descargamos el fichero datasetEspañol.csv
!wget --no-check-certificate https://valencia.inf.um.es/dlpln/datasetEspañol.csv


--2024-11-11 15:07:21--  https://valencia.inf.um.es/dlpln/BM25.py
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3049 (3.0K) [text/x-python]
Saving to: ‘BM25.py’

BM25.py             100%[===================>]   2.98K  --.-KB/s    in 0s      

2024-11-11 15:07:22 (447 MB/s) - ‘BM25.py’ saved [3049/3049]

--2024-11-11 15:07:22--  https://valencia.inf.um.es/dlpln/datasetEspa%C3%B1ol.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1953117 (1.9M) [text/csv]
Saving to: ‘datasetEspañol.csv’

datasetEspañol.csv  100%[===================>]   1.86M  2.22MB/s    in 0.8s    

2024-11-11 15:07:23 (2.22 MB/s) - ‘datasetEspañol.csv’ saved [1953117/1953117]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Importamos del fichero BM25.py
from BM25 import BM25Transformer

In [ ]:
# Leemos los datos del dataset en español de la sesión 2.1
data = pd.read_csv("datasetEspañol.csv",encoding="UTF-8")
data.tail()

,twitter_id,twitter_created_at,tweet,corpus,user,agreement,votes,score,label,__split
5954,1274324047581581316,2020-06-20 16:51:43,No me fío nada! ? en #estadodealarma demostrar...,Estado de alarma nacional (oficial),GuzmanitaMaria,100,1,-1,negative,test
5955,1274321386446733315,2020-06-20 16:41:08,@roldanfj1 @MikiyDuarte @diariocadiz @realDona...,Estado de alarma nacional (oficial),ByChanchi,100,1,-1,negative,train
5956,1274340519271858178,2020-06-20 17:57:10,"Con el fin del #EstadodeAlarma se acaban, tamb...",Estado de alarma nacional (oficial),Javiersilvestre,100,1,-1,negative,val
5957,1274367246979211269,2020-06-20 19:43:22,@horaciorlarreta. @AsisOberdan. @luisnovaresio...,Estado de alarma nacional (oficial),juliodebarna,100,1,-1,negative,train
5958,1274368625047220224,2020-06-20 19:48:51,En la última semana se han registrado 36 falle...,Estado de alarma nacional (oficial),mallorcadiario,100,1,-1,negative,train


## Apartado 1 - Obtener TF del conjunto de texto (Resuelto)

Calculamos la matriz de TF usando la clase CountVectorizer sobre un conjunto de textos.

Se puede consultar información de esta clase en la siguiente URL:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer

Hay que tener en cuenta que esta clase tiene muchos parámetros en su método de creación.

In [ ]:
texto = """La asignatura de DLPLN es una asignatura del máster de Inteligencia Artificial que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos tecnologías de procesamiento del lenguaje natural y deep learning.
"""
texto2 = "No me gusta el chocolate ni las fresas"

texto3 = """El profesor de la asignatura DLPLN es Rafael Valencia García.
"""

# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform([texto,texto2,texto3])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_counts[0])
print(texto2)
print(X_counts[1])
print(texto3)
print(X_counts[2])

#Los tokens de todo el vocabulario se representan con ids que hacen referencia a cada token.
print("Mostramos los items del diccionario")
print(count_vect.vocabulary_.items())
print("Tamaño vocabulario:", str(len(count_vect.vocabulary_.items())))

#Mostramos el código de una palabra determinada
#hay que tener en cuenta que todos los tokens se guardan en minúsculas
palabra_a_buscar="DLPLN"
print("Código de la palabra", palabra_a_buscar, "es:", count_vect.vocabulary_.get(palabra_a_buscar.lower()))

(3, 35)
La asignatura de DLPLN es una asignatura del máster de Inteligencia Artificial que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos tecnologías de procesamiento del lenguaje natural y deep learning.

  (0, 15)	3
  (0, 1)	3
  (0, 3)	5
  (0, 6)	1
  (0, 9)	1
  (0, 31)	1
  (0, 5)	2
  (0, 21)	2
  (0, 14)	1
  (0, 0)	1
  (0, 27)	1
  (0, 29)	1
  (0, 10)	1
  (0, 8)	2
  (0, 32)	1
  (0, 20)	1
  (0, 34)	1
  (0, 30)	1
  (0, 25)	1
  (0, 18)	1
  (0, 22)	1
  (0, 4)	1
  (0, 17)	1
No me gusta el chocolate ni las fresas
  (0, 24)	1
  (0, 19)	1
  (0, 13)	1
  (0, 7)	1
  (0, 2)	1
  (0, 23)	1
  (0, 16)	1
  (0, 11)	1
El profesor de la asignatura DLPLN es Rafael Valencia García.

  (0, 15)	1
  (0, 1)	1
  (0, 3)	1
  (0, 6)	1
  (0, 9)	1
  (0, 7)	1
  (0, 26)	1
  (0, 28)	1
  (0, 33)	1
  (0, 12)	1
Mostramos los items del diccionario
dict_items([('la', 15), ('asignatura', 1), ('de', 3), ('dlpln', 6), ('es', 9), ('una', 31), ('del', 5), ('máster', 21), ('inteligencia', 14), ('artificia

## Apartado 2 Calculamos el TF sin tener en cuenta las stopwords
Para eso hacemos uso del parámetro stop_words de CountVectorizer

In [ ]:
import nltk
# Descargamos las stopwords de NLTK
# Si no tenemos instalado NLTK lo instalamos
# !pip3 install -U nltk
nltk.download('stopwords')
stopwords_sp = nltk.corpus.stopwords.words('spanish')

# Calculamos la matriz de TF usando la función fit_transform quitando las stopwords
count_vect = CountVectorizer(stop_words=stopwords_sp)
# count_vect = CountVectorizer(stop_words=stopwords_sp, min_df=2)
X_counts = count_vect.fit_transform([texto,texto2,texto3])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_counts[0])
print(texto2)
print(X_counts[1])
print(texto3)
print(X_counts[2])

#Los tokens de todo el vocabulario se representan con ids que hacen referencia a cada token.
print("Mostramos los items del diccionario")
print(count_vect.vocabulary_.items())

(3, 22)
La asignatura de DLPLN es una asignatura del máster de Inteligencia Artificial que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos tecnologías de procesamiento del lenguaje natural y deep learning.

  (0, 1)	3
  (0, 4)	1
  (0, 13)	2
  (0, 9)	1
  (0, 0)	1
  (0, 5)	1
  (0, 19)	1
  (0, 12)	1
  (0, 21)	1
  (0, 18)	1
  (0, 15)	1
  (0, 11)	1
  (0, 14)	1
  (0, 3)	1
  (0, 10)	1
No me gusta el chocolate ni las fresas
  (0, 8)	1
  (0, 2)	1
  (0, 6)	1
El profesor de la asignatura DLPLN es Rafael Valencia García.

  (0, 1)	1
  (0, 4)	1
  (0, 16)	1
  (0, 17)	1
  (0, 20)	1
  (0, 7)	1
Mostramos los items del diccionario
dict_items([('asignatura', 1), ('dlpln', 4), ('máster', 13), ('inteligencia', 9), ('artificial', 0), ('estudia', 5), ('universidad', 19), ('murcia', 12), ('vemos', 21), ('tecnologías', 18), ('procesamiento', 15), ('lenguaje', 11), ('natural', 14), ('deep', 3), ('learning', 10), ('gusta', 8), ('chocolate', 2), ('fresas', 6), ('profesor', 16), ('rafael',

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Apartado 3 TF-IDF (Resuelto)
Obtenemos el TF-IDF utilizando la clase TfidfTransformer

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(norm=None)
X_TFIDF = tfidf_transformer.fit_transform(X_counts)

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_TFIDF.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_TFIDF[0])
print(texto2)
print(X_TFIDF[1])
print(texto3)
print(X_TFIDF[2])

(3, 22)
La asignatura de DLPLN es una asignatura del máster de Inteligencia Artificial que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos tecnologías de procesamiento del lenguaje natural y deep learning.

  (0, 0)	0.2142551353942561
  (0, 1)	0.48883965897165205
  (0, 3)	0.2142551353942561
  (0, 4)	0.16294655299055066
  (0, 5)	0.2142551353942561
  (0, 9)	0.2142551353942561
  (0, 10)	0.2142551353942561
  (0, 11)	0.2142551353942561
  (0, 12)	0.2142551353942561
  (0, 13)	0.4285102707885122
  (0, 14)	0.2142551353942561
  (0, 15)	0.2142551353942561
  (0, 18)	0.2142551353942561
  (0, 19)	0.2142551353942561
  (0, 21)	0.2142551353942561
No me gusta el chocolate ni las fresas
  (0, 2)	0.5773502691896257
  (0, 6)	0.5773502691896257
  (0, 8)	0.5773502691896257
El profesor de la asignatura DLPLN es Rafael Valencia García.

  (0, 1)	0.3349067026613031
  (0, 4)	0.3349067026613031
  (0, 7)	0.4403620672313486
  (0, 16)	0.4403620672313486
  (0, 17)	0.4403620672313486
  (0, 20)

## Apartado 4 BM25 (Resuelto)

Calculamos el BM25 que tiene en cuenta tanto la transformación del TF como la normalización de la longitud del documento.

In [ ]:
bm25_transformer = BM25Transformer(k=1.2, b=0.5)
X_BM25 = bm25_transformer.fit_transform(X_counts)

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_BM25.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_BM25[0])
print(texto2)
print(X_BM25[1])
print(texto3)
print(X_BM25[2])

(3, 22)
La asignatura de DLPLN es una asignatura del máster de Inteligencia Artificial que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos tecnologías de procesamiento del lenguaje natural y deep learning.

  (0, 0)	0.24602279916683756
  (0, 1)	0.32743669020217797
  (0, 3)	0.24602279916683756
  (0, 4)	0.18710668011553022
  (0, 5)	0.24602279916683756
  (0, 9)	0.24602279916683756
  (0, 10)	0.24602279916683756
  (0, 11)	0.24602279916683756
  (0, 12)	0.24602279916683756
  (0, 13)	0.3625599145616553
  (0, 14)	0.24602279916683756
  (0, 15)	0.24602279916683756
  (0, 18)	0.24602279916683756
  (0, 19)	0.24602279916683756
  (0, 21)	0.24602279916683756
No me gusta el chocolate ni las fresas
  (0, 2)	0.5773502691896257
  (0, 6)	0.5773502691896257
  (0, 8)	0.5773502691896257
El profesor de la asignatura DLPLN es Rafael Valencia García.

  (0, 1)	0.3349067026613031
  (0, 4)	0.3349067026613031
  (0, 7)	0.4403620672313485
  (0, 16)	0.4403620672313485
  (0, 17)	0.44036206723134

# Apartado 5 Desarrollamos un simple buscador con TFIDF y BM25 (Resuelto)

A continuación vamos a procesar los tuits del archivo de la sesión 2.1 "datosEspañol.csv" y calculamos el TF, el TFIDF y el BM25 de manera similar a como se ha hecho anteriormente.

In [ ]:
# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer(stop_words=stopwords_sp)
X_counts = count_vect.fit_transform(data['tweet'])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos el primer tuit y su correspondientes TF
print(data['tweet'][0])
print(X_counts[0])

(5959, 23922)
Hoy merendola deliciosa! Latte Macchiato Caramelo con Leche Condensada y Gofre! ???. #yomequedoencasa #todovaasalirbien #undiamenos #actitudpositiva #lattemacchiato #gofre #delicious #strong #stronger #smile…
  (0, 11621)	1
  (0, 14830)	1
  (0, 6839)	1
  (0, 13387)	1
  (0, 14082)	1
  (0, 4063)	1
  (0, 13452)	1
  (0, 5362)	1
  (0, 10888)	2
  (0, 23725)	1
  (0, 22067)	1
  (0, 22606)	1
  (0, 1018)	1
  (0, 13388)	1
  (0, 6843)	1
  (0, 21278)	1
  (0, 21279)	1
  (0, 20860)	1


In [ ]:
# El vocabulario que forma los tokens del objeto vectorizer se puede obtener de la siguiente manera
# obtenemos el id del token 'buenosdias' que proviene de un hashtag
count_vect.vocabulary_.get('buenosdias')

3657

In [ ]:
# Calculamos ahora el TFIDF
X_TFIDF = tfidf_transformer.fit_transform(X_counts)
# Mostramos entonces el número de textos y el número de tokens únicos
print(X_TFIDF.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos el primer tuit y su correspondientes TF
print(data['tweet'][0])
print(X_TFIDF[0])

# Calculamos también el BM25
X_BM25 = bm25_transformer.fit_transform(X_counts)
# Mostramos entonces el número de textos y el número de tokens únicos
print(X_BM25.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos el primer tuit y su correspondientes TF
print(data['tweet'][0])
print(X_BM25[0])

(5959, 23922)
Hoy merendola deliciosa! Latte Macchiato Caramelo con Leche Condensada y Gofre! ???. #yomequedoencasa #todovaasalirbien #undiamenos #actitudpositiva #lattemacchiato #gofre #delicious #strong #stronger #smile…
  (0, 1018)	0.22166053291045124
  (0, 4063)	0.24015722738486325
  (0, 5362)	0.24015722738486325
  (0, 6839)	0.24015722738486325
  (0, 6843)	0.2157059272965902
  (0, 10888)	0.4803144547697265
  (0, 11621)	0.09324696322101765
  (0, 13387)	0.24015722738486325
  (0, 13388)	0.24015722738486325
  (0, 13452)	0.21084066025562398
  (0, 14082)	0.24015722738486325
  (0, 14830)	0.24015722738486325
  (0, 20860)	0.2157059272965902
  (0, 21278)	0.24015722738486325
  (0, 21279)	0.24015722738486325
  (0, 22067)	0.1503006371554885
  (0, 22606)	0.17275793273390508
  (0, 23725)	0.049423969305183627
(5959, 23922)
Hoy merendola deliciosa! Latte Macchiato Caramelo con Leche Condensada y Gofre! ???. #yomequedoencasa #todovaasalirbien #undiamenos #actitudpositiva #lattemacchiato #gofre #deli

In [ ]:
# Podemos ver que el tamaño de la matriz de textos y el número total de tokens es el mismo tanto
# para TFIDF como para BM25
print(X_TFIDF.shape) # (Number of tweets, Number of unique words)
print(X_BM25.shape)

(5959, 23922)
(5959, 23922)


Realizamos una consulta cualquiera y la metemos en el string "query" para a continuación calcular la similitud del coseno usando el TF-IDF y el BM25.

In [ ]:
query = "semana santa"
# query = "santa semana"

In [ ]:
# Transformamos la query a TF-ID y sacamos los resultados de la comparación con la función del coseno
# cosine_similarity
query_vec = count_vect.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
query_vec_TFIDF = tfidf_transformer.fit_transform(query_vec)
results = cosine_similarity(X_TFIDF,query_vec_TFIDF).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

# print(query_vec_TFIDF)
# Immprimimos a continuación los primeros 10 resultados ordenados por la similitud obtenida
for i in results.argsort()[-10:][::-1]:
    print(data.iloc[i,2],"--",results[i],"--",i)

Aunque sean unos momentos difíciles, debemos tomarlo con el mayor positivismo posible! La Semana Santa no es sólo salir a beber y a ver procesiones, es mucho más que eso. Disfrutemos de esta semana de una forma verdadera! Feliz Domingo de Ramos y Semana Santa!!! #YoMeQuedoEnCasa -- 0.5341953468689503 -- 804
Es tiempo de recogimiento ?. Esta Semana Santa no se sale. #SemanaSanta #YoMeQuedoEnCasa #QuedateEnCasa -- 0.4618900044681435 -- 4863
Buen dia amigos espero que estén todos bien!!! Feliz inicio de semana arriba ese ánimo todo lo mejor, que sea una Semana Santa de bendiciones!!! A cuidarse . #QuedateEnCasa -- 0.450087834395828 -- 31
Esta Semana Santa quiero salvar vidas: #YoMeQuedoEnCasa . Si te vas de “vacaciones”, ¿cuántas muertes caerán sobre tus espaldas la semana que viene...?. #QuedateEnTuCasa ?? -- 0.40809989875165065 -- 1372
¡Feliz Semana Santa! Aunque sea en casa ?. #AyudaADomicilio #Mayores #QuéDateEnCasa #Santander #Cantabria -- 0.4044128930022516 -- 1967
La procesión de c

In [ ]:
# Obtenemos ahora los resultados usando el BM25
query_vec = count_vect.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
query_vec_BM25 = bm25_transformer.fit_transform(query_vec)
results = cosine_similarity(X_BM25,query_vec_BM25).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

# Immprimimos a continuación los primeros 10 resultados ordenados por la similitud obtenida
for i in results.argsort()[-10:][::-1]:
    print(data.iloc[i,2],"--",results[i],"--",i)

Es tiempo de recogimiento ?. Esta Semana Santa no se sale. #SemanaSanta #YoMeQuedoEnCasa #QuedateEnCasa -- 0.4618900044681435 -- 4863
¡Feliz Semana Santa! Aunque sea en casa ?. #AyudaADomicilio #Mayores #QuéDateEnCasa #Santander #Cantabria -- 0.4044128930022516 -- 1967
La procesión de carros de compra que estoy viendo desde mi balcón sirve como Semana Santa? #YoMeQuedoEnCasa -- 0.38389199667649315 -- 156
Buen dia amigos espero que estén todos bien!!! Feliz inicio de semana arriba ese ánimo todo lo mejor, que sea una Semana Santa de bendiciones!!! A cuidarse . #QuedateEnCasa -- 0.38070852809993044 -- 31
Aunque sean unos momentos difíciles, debemos tomarlo con el mayor positivismo posible! La Semana Santa no es sólo salir a beber y a ver procesiones, es mucho más que eso. Disfrutemos de esta semana de una forma verdadera! Feliz Domingo de Ramos y Semana Santa!!! #YoMeQuedoEnCasa -- 0.37318611827279735 -- 804
Último día de teletrabajo hasta después de Semana Santa ???. Gracias equipo TeSe

# Apartado 6 Modificamos el buscador anterior para trabajar con bigramas o trigramas

Configuraremos el CountVectorizer para trabajar con unigramas, bigramas y trigramas y probamos distintas consultas.

In [ ]:
# Calculamos la matriz de TF usando la función fit_transform
###########################################################
# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer(stop_words=stopwords_sp, ngram_range=(1,3))
X_counts = count_vect.fit_transform(data['tweet'])

# Calculamos ahora el TFIDF
X_TFIDF = tfidf_transformer.fit_transform(X_counts)

# Calculamos también el BM25
X_BM25 = bm25_transformer.fit_transform(X_counts)

X_counts.shape

In [ ]:
query = "semana santa"
# query = "santa semana"

In [ ]:
# Transformamos la query a TF-ID y sacamos los resultados de la comparación con la función del coseno
# cosine_similarity
query_vec = count_vect.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
query_vec_TFIDF = tfidf_transformer.fit_transform(query_vec)
results = cosine_similarity(X_TFIDF,query_vec_TFIDF).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

# Immprimimos a continuación los primeros 10 resultados ordenados por la similitud obtenida
for i in results.argsort()[-10:][::-1]:
    print(data.iloc[i,2],"--",results[i],"--",i)

In [ ]:
# Obtenemos ahora los resultados usando el BM25
query_vec = count_vect.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
query_vec_BM25 = bm25_transformer.fit_transform(query_vec)
results = cosine_similarity(X_BM25,query_vec_BM25).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

# Immprimimos a continuación los primeros 10 resultados ordenados por la similitud obtenida
for i in results.argsort()[-10:][::-1]:
    print(data.iloc[i,2],"--",results[i],"--",i)

## Apartado 7 Calculamos la similitud de varios textos con TF-IDF y BM25 (Resuelto)

Vamos a calcular la similitud de el primer texto con respecto a los demás usando la similitud del coseno.

In [ ]:
# Definimos un conjunto de textos
textos=['El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. \
Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje.\
Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'El procesamiento del lenguaje natural (NLP, por sus siglas en inglés) es una rama de la inteligencia artificial que ayuda a las computadoras a entender, interpretar y manipular el lenguaje humano. \
NLP toma elementos prestados de muchas disciplinas, incluyendo la ciencia de la computación y la lingüística computacional, en su afán por cerrar la brecha entre la comunicación humana y el entendimiento de las computadoras."""], """El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje. Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'La lingüística computacional es un campo interdisciplinario que se ocupa del desarrollo de formalismos del funcionamiento del lenguaje natural, tales que puedan ser transformados en programas ejecutables para un ordenador. \
Dicho desarrollo se sitúa entre el modelado basado en reglas y el modelado estadístico del lenguaje natural desde una perspectiva computacional, y en él participan lingüistas e informáticos especializados en inteligencia artificial, psicólogos cognoscitivos y expertos en lógica, entre otros.'
, 'El aprendizaje automático es un tipo de inteligencia artificial (AI) que proporciona a las computadoras la capacidad de aprender, sin ser programadas explícitamente. El aprendizaje automático se centra en el desarrollo de programas informáticos que pueden cambiar cuando se exponen a nuevos datos.'
, 'El  aprendizaje profundo es un tema que cada vez adquiere mayor relevancia en el campo de la inteligencia artificial (IA). Siendo una subcategoría del aprendizaje automático, el aprendizaje profundo trata del uso de redes neuronales para mejorar cosas tales como el reconocimiento de voz, la visión por ordenador y el procesamiento del lenguaje natural. \
Rápidamente se está convirtiendo en uno de los campos más solicitados en informática. \
En los últimos años, el aprendizaje profundo ha ayudado a lograr avances en áreas tan diversas como la percepción de objetos, el procesamiento del lenguaje natural y el reconocimiento de voz (todas ellas áreas especialmente complejas para los investigadores en IA).'
]

# Calculamos la similitud usando TFIDF
count_vect = CountVectorizer(stop_words=stopwords_sp)
X_counts = count_vect.fit_transform(textos)

# Calculamos ahora el TFIDF
tfidf_transformer = TfidfTransformer()
X_TFIDF = tfidf_transformer.fit_transform(X_counts)

# Calculamos también el BM25
BM25_transformer = BM25Transformer(k=1.2,b=0.75)
X_BM25 = BM25_transformer.fit_transform(X_counts)

# Calculamos la similitud de los documentos con el coseno para TFIDF
results = cosine_similarity(X_TFIDF[1::],X_TFIDF[0]).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
print("Similitud de textos con TFIDF:", results)

# Calculamos la similitud de los textos con el coseno para BM25
results = cosine_similarity(X_BM25[1::],X_TFIDF[0]).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
print("Similitud de textos con TFIDF:",results)



## Apartado 8 Cambiando a character n-grams
Podemos usar en vez de palabras (words), character n-grams para crear el vocabulario. Lo vemos con el primer ejemplo

In [ ]:
texto = """La asignatura de DLPLN es una asignatura del máster de Inteligencia Artificial que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos tecnologías de procesamiento del lenguaje natural y deep learning.
"""
texto2 = "No me gusta el chocolate ni las fresas"

texto3 = """El profesor de la asignatura DLPLN es Rafael Valencia García.
"""
# Calculamos la matriz de TF usando la función fit_transform
############################################################
count_vect = CountVectorizer(analyzer='char', ngram_range=(3,5), min_df=2)
X_counts = count_vect.fit_transform([texto,texto2,texto3])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_counts[0])
print(texto2)
print(X_counts[1])
print(texto3)
print(X_counts[2])

#Los tokens de todo el vocabulario se representan con ids que hacen referencia a cada token.
print("Mostramos los items del diccionario")
print(count_vect.vocabulary_.items())
print("Tamaño vocabulario:", str(len(count_vect.vocabulary_.items())))

#Mostramos el código de una palabra determinada
#hay que tener en cuenta que todos los tokens se guardan en minúsculas
palabra_a_buscar="DLP"
print("Código de la palabra", palabra_a_buscar, "es:", count_vect.vocabulary_.get(palabra_a_buscar.lower()))